In [1]:
# Import Modules and Packages. 
import numpy as np
import pandas as pd
import hvplot.pandas
import seaborn as sns

from pathlib import Path

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn import preprocessing 

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

Linear Regression & Predictions using Multiple Variables, Supervised Machine Learning Methods.

In [4]:
# Read in cleansed data file 'data_drop.csv' from AWS S3 Bucket

url="https://project-4-group-6-air-quality.s3.us-east-2.amazonaws.com/data_drop.csv"
air3_data_df = pd.read_csv(
    url,
    sep=',',
    encoding='utf-8',
)

# Display sample data
air3_data_df.head()


,Unnamed: 0,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM
0,0,2013,3,1,0,9.0,9.0,3.0,17.0,300.0,89.0,-0.5,1024.5,-21.4,0.0,NNW,5.7
1,1,2013,3,1,1,4.0,4.0,3.0,16.0,300.0,88.0,-0.7,1025.1,-22.1,0.0,NW,3.9
2,2,2013,3,1,5,4.0,4.0,9.0,25.0,300.0,78.0,-2.4,1027.5,-21.3,0.0,NW,2.4
3,3,2013,3,1,6,5.0,5.0,10.0,29.0,400.0,67.0,-2.5,1028.2,-20.4,0.0,NW,2.2
4,4,2013,3,1,7,3.0,6.0,12.0,40.0,400.0,52.0,-1.4,1029.5,-20.4,0.0,NNW,3.0


In [5]:
type(air3_data_df)

pandas.core.frame.DataFrame

In [7]:
# Rename Unnamed:0 column
air3_data_df.rename(columns ={"Unnamed: 0":"Number"})

,Number,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM
0,0,2013,3,1,0,9.0,9.0,3.0,17.0,300.0,89.0,-0.5,1024.5,-21.4,0.0,NNW,5.7
1,1,2013,3,1,1,4.0,4.0,3.0,16.0,300.0,88.0,-0.7,1025.1,-22.1,0.0,NW,3.9
2,2,2013,3,1,5,4.0,4.0,9.0,25.0,300.0,78.0,-2.4,1027.5,-21.3,0.0,NW,2.4
3,3,2013,3,1,6,5.0,5.0,10.0,29.0,400.0,67.0,-2.5,1028.2,-20.4,0.0,NW,2.2
4,4,2013,3,1,7,3.0,6.0,12.0,40.0,400.0,52.0,-1.4,1029.5,-20.4,0.0,NNW,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382163,382163,2017,2,28,19,11.0,11.0,2.0,2.0,200.0,99.0,11.7,1008.9,-13.3,0.0,NNE,1.3
382164,382164,2017,2,28,20,13.0,13.0,2.0,2.0,200.0,101.0,10.9,1009.0,-14.0,0.0,N,2.1
382165,382165,2017,2,28,21,9.0,14.0,2.0,2.0,200.0,102.0,9.5,1009.4,-13.0,0.0,N,1.5
382166,382166,2017,2,28,22,10.0,12.0,2.0,2.0,200.0,97.0,7.8,1009.6,-12.6,0.0,NW,1.4


In [13]:
air3_df=air3_data_df.drop(columns=["Unnamed: 0",'year','month','day','hour','wd'])
air3_df

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
0,9.0,9.0,3.0,17.0,300.0,89.0,-0.5,1024.5,-21.4,0.0,5.7
1,4.0,4.0,3.0,16.0,300.0,88.0,-0.7,1025.1,-22.1,0.0,3.9
2,4.0,4.0,9.0,25.0,300.0,78.0,-2.4,1027.5,-21.3,0.0,2.4
3,5.0,5.0,10.0,29.0,400.0,67.0,-2.5,1028.2,-20.4,0.0,2.2
4,3.0,6.0,12.0,40.0,400.0,52.0,-1.4,1029.5,-20.4,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...
382163,11.0,11.0,2.0,2.0,200.0,99.0,11.7,1008.9,-13.3,0.0,1.3
382164,13.0,13.0,2.0,2.0,200.0,101.0,10.9,1009.0,-14.0,0.0,2.1
382165,9.0,14.0,2.0,2.0,200.0,102.0,9.5,1009.4,-13.0,0.0,1.5
382166,10.0,12.0,2.0,2.0,200.0,97.0,7.8,1009.6,-12.6,0.0,1.4


In [14]:
air3_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382168 entries, 0 to 382167
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   PM2.5   382168 non-null  float64
 1   PM10    382168 non-null  float64
 2   SO2     382168 non-null  float64
 3   NO2     382168 non-null  float64
 4   CO      382168 non-null  float64
 5   O3      382168 non-null  float64
 6   TEMP    382168 non-null  float64
 7   PRES    382168 non-null  float64
 8   DEWP    382168 non-null  float64
 9   RAIN    382168 non-null  float64
 10  WSPM    382168 non-null  float64
dtypes: float64(11)
memory usage: 32.1 MB


In [16]:
print(air3_df.columns)

Index(['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP',
       'RAIN', 'WSPM'],
      dtype='object')


In [18]:
# creating feature variables 
X = air3_df.drop('O3',axis= 1) 
y = air3_df['O3'] 
print(X) 
print(y)

        PM2.5  PM10   SO2   NO2     CO  TEMP    PRES  DEWP  RAIN  WSPM
0         9.0   9.0   3.0  17.0  300.0  -0.5  1024.5 -21.4   0.0   5.7
1         4.0   4.0   3.0  16.0  300.0  -0.7  1025.1 -22.1   0.0   3.9
2         4.0   4.0   9.0  25.0  300.0  -2.4  1027.5 -21.3   0.0   2.4
3         5.0   5.0  10.0  29.0  400.0  -2.5  1028.2 -20.4   0.0   2.2
4         3.0   6.0  12.0  40.0  400.0  -1.4  1029.5 -20.4   0.0   3.0
...       ...   ...   ...   ...    ...   ...     ...   ...   ...   ...
382163   11.0  11.0   2.0   2.0  200.0  11.7  1008.9 -13.3   0.0   1.3
382164   13.0  13.0   2.0   2.0  200.0  10.9  1009.0 -14.0   0.0   2.1
382165    9.0  14.0   2.0   2.0  200.0   9.5  1009.4 -13.0   0.0   1.5
382166   10.0  12.0   2.0   2.0  200.0   7.8  1009.6 -12.6   0.0   1.4
382167   13.0  16.0   4.0   9.0  500.0   7.0  1009.4 -12.2   0.0   1.9

[382168 rows x 10 columns]
0          89.0
1          88.0
2          78.0
3          67.0
4          52.0
          ...  
382163     99.0
382164  

Scatter Plot with Ozone (O3) and Temperature (TEMP)

In [19]:
# creating train and test sets 
X_train, X_test, y_train, y_test = train_test_split( 
    X, y, test_size=0.3, random_state=101) 

In [20]:
# creating a regression model 
model = LinearRegression() 

In [21]:
# fitting the model 
model.fit(X_train,y_train)

LinearRegression()

In [22]:
# making predictions 
predictions = model.predict(X_test)

In [23]:
# model evaluation 
print( 
  'mean_squared_error : ', mean_squared_error(y_test, predictions)) 
print( 
  'mean_absolute_error : ', mean_absolute_error(y_test, predictions)) 

mean_squared_error :  1378.584450481671
mean_absolute_error :  27.07180456217113


In [ ]:
# Create a scatter plot
air_plot = air_df.hvplot.scatter(
    x="TEMP",
    y="O3",
    title="Expected Ozone Measures based on Temperature"
)
air_plot

Linear Regression Model to predict Ozone based on Temperature.

In [ ]:
# Create the X set by using the `reshape` function to format the TEMP data as a single column array.
X = air_df["TEMP"].values.reshape(-1, 1)

# Display sample data
X[:5]

In [ ]:
# Create an array for the dependent variable y with the O3 data
y = air_df["O3"]

Linear Regression Model with SciKit-learn

In [ ]:
# Create a model with scikit-learn
model = LinearRegression()

In [ ]:
# Fit the data into the model
model.fit(X, y)

In [ ]:
# Display the slope
print(f"Model's slope: {model.coef_}")

In [ ]:
# Display the y-intercept
print(f"Model's y-intercept: {model.intercept_}")

In [ ]:
# Display the model's best fit line formula
print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]}X")

Plot of Best Fit Line for the Prediction Model

In [ ]:
# Make predictions using the X set
predicted_y_values = model.predict(X)

In [ ]:
# Create a copy of the original data
air_ozone_predicted = air_df.copy()

# Add a column with the predicted sales values
air_ozone_predicted["Ozone_predicted"] = predicted_y_values

# Display sample data
air_ozone_predicted.head()

In [ ]:
# Create a line plot of ads versus the predicted sales values
best_fit_line = air_ozone_predicted.hvplot.line(
    x = "TEMP",
    y = "Ozone_predicted",
    color = "orange"
)
best_fit_line

In [ ]:
# Superpose the original data and the best fit line
air_plot * best_fit_line

Manual Predictions

In [ ]:
# Display the formula to predict the sales with 100 ads
print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]} * 100")

# Predict the sales with 100 ads
y_100 = model.intercept_ + model.coef_[0] * 100

# Display the prediction
print(f"Predicted Ozone (O3) metrics {y_100:.2f}")

Predictions Using the `Predict` Function

In [ ]:
# Create an array to predict ozone levels for -0.5, -1.0, -1.5, -2.0, and -2.5 Temps
X_temps = np.array([-0.5, -1.0, -1.5, -2.0, -2.5])

# Format the array as a one-column array
X_temps = X_temps.reshape(-1,1)

# Display sample data
X_temps

In [ ]:
# Predict ozone for temp values
predicted_ozone_temp = model.predict(X_temps)

In [ ]:
# Create a DataFrame for the predicted ozone levels
df_predicted_ozone_temp = pd.DataFrame(
    {
        "temp": X_temps.reshape(1, -1)[0],
        "predicted_ozone": predicted_ozone_temp
    }
)

# Display data
df_predicted_ozone_temp

Evaluate the Linear Regression Model

In [ ]:
# Compute metrics for the linear regression model: score, r2, mse, rmse, std
score = model.score(X, y, sample_weight=None)
r2 = r2_score(y, predicted_y_values)
mse = mean_squared_error(y, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(y)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

---